<a href="https://www.kaggle.com/code/tanakataro73/kaggle-book-morohashi-07-home-credit-default-risk?scriptVersionId=172282536" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv


In [2]:
import re
import pickle
import gc

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [3]:
application_train = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
print(application_train.shape)
application_train.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            pass

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
application_train = reduce_mem_usage(application_train)

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%


In [6]:
# 学習を関数化
def train_lgb(
    input_x,
    input_y,
    input_id,
    params,
    list_nfold=[0,1,2,3,4],
    n_splits=5,
):
    train_oof = np.zeros(len(input_x))
    metrics = []
    imp = pd.DataFrame()
    
    # cross-validation
    cv = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123).split(input_x, input_y))
    for nfold in list_nfold:
        print("-"*20, nfold, "-"*20)
        
        # make dataset
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr, id_tr = input_x.loc[idx_tr, :], input_y[idx_tr], input_id.loc[idx_tr, :]
        x_va, y_va, id_va = input_x.loc[idx_va, :], input_y[idx_va], input_id.loc[idx_va, :]
        print(x_tr.shape, x_va.shape)
        
        # train
        model = lgb.LGBMClassifier(**params)
        model.fit(
            x_tr,
            y_tr,
            eval_set=[(x_tr, y_tr), (x_va, y_va)],
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=True)],
        )
        
        fname_lgb = "model_lgb_fold{}.pickle".format(nfold)
        with open(fname_lgb, "wb") as f:
            pickle.dump(model, f, protocol=4)
        
        # eval
        # 学習データの推論値とROC
        # 目的変数が1になる確率
        y_tr_pred = model.predict_proba(x_tr)[:,1]
        metric_tr = roc_auc_score(y_tr, y_tr_pred)
        # 検証データの推論値とROC
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        metrics.append([nfold, metric_tr, metric_va])
        print("[auc] tr:{:.4f}, va:{:.4f}".format(metric_tr, metric_va))
        
        # oof
        train_oof[idx_va] = y_va_pred
        
        # importance
        _imp = pd.DataFrame({
            "col": input_x.columns,
            "imp": model.feature_importances_,
            "nfold": nfold,
        })
        imp = pd.concat([imp, _imp])
    
    print("-"*20, "result", "-"*20)
    
    # metric
    metrics = np.array(metrics)
    print(metrics)
    print("[cv] tr:{:.4f}+-{:.4f}, va:{:.4f}+-{:.4f}".format(
        metrics[:,1].mean(), metrics[:,1].std(),
        metrics[:,2].mean(), metrics[:,2].std(),
    ))
    print("[oof] {:.4f}".format(
        roc_auc_score(input_y, train_oof)
    ))

    # oof
    train_oof = pd.concat(
        [
            input_id,
            pd.DataFrame({"pred": train_oof})
        ],
        axis=1,
    )
    
    # importance
    imp = imp.groupby("col")["imp"].agg(["mean", "std"]).reset_index(drop=False)
    imp.columns = ["col", "imp", "imp_std"]
    
    return train_oof, imp, metrics

In [7]:
application_test = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
application_test = reduce_mem_usage(application_test)

Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 14.60 MB
Decreased by 67.6%


In [8]:
# foldごとに学習モデルを読み込み推論、結果を記録
def predict_lbg(
    input_x,
    input_id,
    list_nfold=[0,1,2,3,4]
):
    pred = np.zeros((len(input_x), len(list_nfold)))
    for nfold in list_nfold:
        print("-"*20, nfold, "-"*20)
        fname_lgb = "model_lgb_fold{}.pickle".format(nfold)
        with open(fname_lgb, "rb") as f:
            model = pickle.load(f)
        pred[:, nfold] = model.predict_proba(input_x)[:, 1]
    
    pred = pd.concat(
        [
            input_id,
            pd.DataFrame({"pred": pred.mean(axis=1)}),
        ],
        axis=1,
    )
    
    print("Done.")
    return pred

# 7.4 特徴量エンジニアリング(feature engineering)

In [9]:
display(application_train["DAYS_EMPLOYED"].value_counts())

DAYS_EMPLOYED
 365243    55374
-200         156
-224         152
-230         151
-199         151
           ...  
-13961         1
-11827         1
-10176         1
-9459          1
-8694          1
Name: count, Length: 12574, dtype: int64

In [10]:
print("正の値の割合: {:.4f}".format(
    (application_train["DAYS_EMPLOYED"] > 0).mean()
))

正の値の割合: 0.1801


In [11]:
print("正の値の個数: {}".format(
    (application_train["DAYS_EMPLOYED"] > 0).sum()
))

正の値の個数: 55374


In [12]:
# 「不明」や「未就労」が「365243」になっているもよう 欠損値とする
application_train["DAYS_EMPLOYED"] = application_train["DAYS_EMPLOYED"].replace(365243, np.nan)

In [13]:
# 仮説1 所得が同じでも、家族が多い方が貸倒れしやすそう
# 所得を世帯人数で割る
application_train["INCOME_div_PERSON"] = application_train["AMT_INCOME_TOTAL"] / application_train["CNT_FAM_MEMBERS"]

In [14]:
# 仮説2 所得が同じでも、就労期間が短い方が優秀で貸倒しにくそう(短時間で大金を稼げる)
# 所得を就労期間で割る
application_train["INCOME_div_EMPLOYED"] = application_train["AMT_INCOME_TOTAL"] / application_train["DAYS_EMPLOYED"]

In [15]:
# 仮説3 外部データソースからの評価が平均的に高い方が貸倒しにくそう
application_train["EXT_SOURCE_mean"] = application_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis=1)
application_train["EXT_SOURCE_max"] = application_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].max(axis=1)
application_train["EXT_SOURCE_min"] = application_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].min(axis=1)
application_train["EXT_SOURCE_std"] = application_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].std(axis=1)
application_train["EXT_SOURCE_count"] = application_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].notnull().sum(axis=1)

In [16]:
display(application_train["DAYS_BIRTH"].value_counts())

DAYS_BIRTH
-13749    43
-13481    42
-18248    41
-10020    41
-15771    40
          ..
-24898     1
-24868     1
-8017      1
-25098     1
-25061     1
Name: count, Length: 17460, dtype: int64

In [17]:
# 仮説4 年齢に占める就労期間が長いほうが、貸倒しにくそう
application_train["DAYS_EMPLOYED_div_BIRTH"] = application_train["DAYS_EMPLOYED"] / application_train["DAYS_BIRTH"]

In [18]:
# 年金支払額を所得で割った値
application_train["ANNUITY_div_INCOME"] = application_train["AMT_ANNUITY"] / application_train["AMT_INCOME_TOTAL"]

In [19]:
# 年金支払額を借入金で割った値
application_train["ANNUITY_div_CREDIT"] = application_train["AMT_ANNUITY"] / application_train["AMT_CREDIT"]

In [20]:
# 推論用データ作成
application_test["DAYS_EMPLOYED"] = application_test["DAYS_EMPLOYED"].replace(365243, np.nan)

application_test["INCOME_div_PERSON"] = application_test["AMT_INCOME_TOTAL"] / application_test["CNT_FAM_MEMBERS"]
application_test["INCOME_div_EMPLOYED"] = application_test["AMT_INCOME_TOTAL"] / application_test["DAYS_EMPLOYED"]
application_test["EXT_SOURCE_mean"] = application_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis=1)
application_test["EXT_SOURCE_max"] = application_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].max(axis=1)
application_test["EXT_SOURCE_min"] = application_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].min(axis=1)
application_test["EXT_SOURCE_std"] = application_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].std(axis=1)
application_test["EXT_SOURCE_count"] = application_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].notnull().sum(axis=1)
application_test["DAYS_EMPLOYED_div_BIRTH"] = application_test["DAYS_EMPLOYED"] / application_test["DAYS_BIRTH"]
application_test["ANNUITY_div_INCOME"] = application_test["AMT_ANNUITY"] / application_test["AMT_INCOME_TOTAL"]
application_test["ANNUITY_div_CREDIT"] = application_test["AMT_ANNUITY"] / application_test["AMT_CREDIT"]

# 7.4.2 特徴量エンジニアリング: POS_CASH_balance.csv

In [21]:
pos = pd.read_csv("/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv")
pos = reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 238.45 MB
Decreased by 60.9%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [22]:
pos["SK_ID_CURR"].value_counts()

SK_ID_CURR
265042    295
172612    247
309133    246
127659    245
185185    245
         ... 
406557      1
240277      1
177557      1
251697      1
144536      1
Name: count, Length: 337252, dtype: int64

In [23]:
pos["NAME_CONTRACT_STATUS"].value_counts()

NAME_CONTRACT_STATUS
Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: count, dtype: int64

In [24]:
pos["SK_DPD"].value_counts()

SK_DPD
0       9706131
1         21872
2         17358
3         14403
4         12350
         ...   
3161          1
3217          1
3323          1
3399          1
3082          1
Name: count, Length: 3400, dtype: int64

In [25]:
pos_ohe = pd.get_dummies(pos, columns=["NAME_CONTRACT_STATUS"], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
col_ohe

['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Amortized debt',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CONTRACT_STATUS_nan']

In [26]:
# SK_ID_CURR をキーに集約
pos_ohe_agg = pos_ohe.groupby("SK_ID_CURR").agg(
    {
        # 数値
        "MONTHS_BALANCE": ["mean", "std", "min", "max"],
        "CNT_INSTALMENT": ["mean", "std", "min", "max"],
        "CNT_INSTALMENT_FUTURE": ["mean", "std", "min", "max"],
        "SK_DPD": ["mean", "std", "min", "max"],
        "SK_DPD_DEF": ["mean", "std", "min", "max"],
        # カテゴリ変数(one hot encoding)
        'NAME_CONTRACT_STATUS_Active': ["mean"],
        'NAME_CONTRACT_STATUS_Amortized debt': ["mean"],
        'NAME_CONTRACT_STATUS_Approved': ["mean"],
        'NAME_CONTRACT_STATUS_Canceled': ["mean"],
        'NAME_CONTRACT_STATUS_Completed': ["mean"],
        'NAME_CONTRACT_STATUS_Demand': ["mean"],
        'NAME_CONTRACT_STATUS_Returned to the store': ["mean"],
        'NAME_CONTRACT_STATUS_Signed': ["mean"],
        'NAME_CONTRACT_STATUS_XNA': ["mean"],
        'NAME_CONTRACT_STATUS_nan': ["mean"],
        # IDのレコード数とユニーク数
        "SK_ID_PREV": ["count", "nunique"],
    }
)

In [27]:
pos_ohe_agg.head()

MONTHS_BALANCE                    CNT_INSTALMENT                  \
                     mean        std min max           mean       std   min   
SK_ID_CURR                                                                    
100001         -72.555556  20.863312 -96 -53       4.000000  0.000000   4.0   
100002         -10.000000   5.627314 -19  -1      24.000000  0.000000  24.0   
100003         -43.785714  24.640162 -77 -18      10.107142  2.806597   6.0   
100004         -25.500000   1.290994 -27 -24       3.750000  0.500000   3.0   
100005         -20.000000   3.316625 -25 -15      11.700000  0.948683   9.0   

                 CNT_INSTALMENT_FUTURE            ...  \
             max                  mean       std  ...   
SK_ID_CURR                                        ...   
100001       4.0              1.444444  1.424001  ...   
100002      24.0             15.000000  5.627314  ...   
100003      12.0              5.785714  3.842811  ...   
100004       4.0              2.250000  1.707825  ...   
100005      12.0              7.200000  3.614784  ...   

           NAME_CONTRACT_STATUS_Approved NAME_CONTRACT_STATUS_Canceled  \
                                    mean                          mean   
SK_ID_CURR                                                               
100001                               0.0                           0.0   
100002                               0.0                           0.0   
100003                               0.0                           0.0   
100004                               0.0                           0.0   
100005                               0.0                           0.0   

           NAME_CONTRACT_STATUS_Completed NAME_CONTRACT_STATUS_Demand  \
                                     mean                        mean   
SK_ID_CURR                                                              
100001                           0.222222                         0.0   
100002                           0.000000                         0.0   
100003                           0.071429                         0.0   
100004                           0.250000                         0.0   
100005                           0.090909                         0.0   

           NAME_CONTRACT_STATUS_Returned to the store  \
                                                 mean   
SK_ID_CURR                                              
100001                                            0.0   
100002                                            0.0   
100003                                            0.0   
100004                                            0.0   
100005                                            0.0   

           NAME_CONTRACT_STATUS_Signed NAME_CONTRACT_STATUS_XNA  \
                                  mean                     mean   
SK_ID_CURR                                                        
100001                        0.000000                      0.0   
100002                        0.000000                      0.0   
100003                        0.000000                      0.0   
100004                        0.000000                      0.0   
100005                        0.090909                      0.0   

           NAME_CONTRACT_STATUS_nan SK_ID_PREV          
                               mean      count nunique  
SK_ID_CURR                                              
100001                          0.0          9       2  
100002                          0.0         19       1  
100003                          0.0         28       3  
100004                          0.0          4       1  
100005                          0.0         11       1  

[5 rows x 32 columns]

In [28]:
pos_ohe_agg.columns

MultiIndex([(                            'MONTHS_BALANCE',    'mean'),
            (                            'MONTHS_BALANCE',     'std'),
            (                            'MONTHS_BALANCE',     'min'),
            (                            'MONTHS_BALANCE',     'max'),
            (                            'CNT_INSTALMENT',    'mean'),
            (                            'CNT_INSTALMENT',     'std'),
            (                            'CNT_INSTALMENT',     'min'),
            (                            'CNT_INSTALMENT',     'max'),
            (                     'CNT_INSTALMENT_FUTURE',    'mean'),
            (                     'CNT_INSTALMENT_FUTURE',     'std'),
            (                     'CNT_INSTALMENT_FUTURE',     'min'),
            (                     'CNT_INSTALMENT_FUTURE',     'max'),
            (                                    'SK_DPD',    'mean'),
            (                                    'SK_DPD',     'std'),
      

In [29]:
pos_ohe_agg.columns = [i + "_" + j for i, j in pos_ohe_agg.columns]

In [30]:
pos_ohe_agg.head()

,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,CNT_INSTALMENT_FUTURE_std,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,1.424001,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,5.627314,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,3.842811,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,1.707825,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,3.614784,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [31]:
pos_ohe_agg.shape

(337252, 32)

In [32]:
# SK_ID_CURR をキーに結合
df_train = pd.merge(application_train, pos_ohe_agg, on="SK_ID_CURR", how="left")
print(df_train.shape)
df_train.head()

(307511, 164)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [33]:
# 推論用データセット作成
df_test = pd.merge(application_test, pos_ohe_agg, on="SK_ID_CURR", how="left")
print(df_test.shape)
df_test.head()

(48744, 163)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


# 7.5 モデルチューニング(ハイパーパラメータチューニング)

In [34]:
import optuna

In [36]:
# 学習用データセット再作成
x_train = df_train.drop(columns=["SK_ID_CURR", "TARGET"])
y_train = df_train["TARGET"]
id_train = df_train[["SK_ID_CURR"]]

for col in x_train.columns:
    if x_train[col].dtype == "O":
        x_train[col] = x_train[col].astype("category")

In [35]:
# 探索しないパラメータ
params_base = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1, # only show fatal error
    "learning_rate": 0.05,
    "n_estimators": 100000,
    "bagging_freq": 1,
}

In [41]:
# 目的関数
def objective(trial):
    params_tuning = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 200),
        "min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True),
    }
    params_tuning.update(params_base)
    
    # モデル学習・評価
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    
    # 高速化のため、1つめのfoldのみとする
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        
        model = lgb.LGBMClassifier(**params_tuning)
        model.fit(
            x_tr,
            y_tr,
            eval_set=[(x_tr, y_tr), (x_va, y_va)],
            callbacks=[lgb.early_stopping(stopping_rounds=100)],
        )

        y_va_pred = model.predict_proba(x_va)[:, 1]
        # 評価指標はAUC
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)
    
    metrics = np.mean(list_metrics)
    return metrics

In [42]:
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2024-04-16 07:11:43,196] A new study created in memory with name: no-name-85fd8b41-6d6a-4ce0-8a0e-206ad0427a93


Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[302]	training's auc: 0.895607	valid_1's auc: 0.771015


[I 2024-04-16 07:16:59,140] Trial 3 finished with value: 0.7710151754915916 and parameters: {'num_leaves': 99, 'min_child_samples': 51, 'min_sum_hessian_in_leaf': 0.008282860349569926, 'feature_fraction': 0.9222969979023155, 'bagging_fraction': 0.5845496189097177, 'lambda_l1': 0.8638820891148515, 'lambda_l2': 14.110618680999176}. Best is trial 3 with value: 0.7710151754915916.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[695]	training's auc: 0.834243	valid_1's auc: 0.773585


[I 2024-04-16 07:18:52,735] Trial 2 finished with value: 0.7735854579637853 and parameters: {'num_leaves': 14, 'min_child_samples': 42, 'min_sum_hessian_in_leaf': 0.00863193274837094, 'feature_fraction': 0.8624096765986273, 'bagging_fraction': 0.9855188869800415, 'lambda_l1': 0.0262953506177079, 'lambda_l2': 0.4794870462552836}. Best is trial 2 with value: 0.7735854579637853.


...

[I 2024-04-16 09:02:14,056] Trial 47 finished with value: 0.7742322188319047 and parameters: {'num_leaves': 9, 'min_child_samples': 85, 'min_sum_hessian_in_leaf': 0.00023943659114182496, 'feature_fraction': 0.9079139766495278, 'bagging_fraction': 0.8521824824533376, 'lambda_l1': 0.45845379786349794, 'lambda_l2': 0.4161749543929318}. Best is trial 38 with value: 0.7755235817655564.


Early stopping, best iteration is:
[1131]	training's auc: 0.81869	valid_1's auc: 0.775385


[I 2024-04-16 09:02:32,085] Trial 45 finished with value: 0.7753846204267539 and parameters: {'num_leaves': 8, 'min_child_samples': 92, 'min_sum_hessian_in_leaf': 0.00043908540028208973, 'feature_fraction': 0.8756737659849709, 'bagging_fraction': 0.867696205121588, 'lambda_l1': 0.4874515105377353, 'lambda_l2': 6.3278894446080525}. Best is trial 38 with value: 0.7755235817655564.


Early stopping, best iteration is:
[837]	training's auc: 0.83825	valid_1's auc: 0.7741


[I 2024-04-16 09:03:22,506] Trial 48 finished with value: 0.77410013288748 and parameters: {'num_leaves': 19, 'min_child_samples': 58, 'min_sum_hessian_in_leaf': 0.0004426438339753066, 'feature_fraction': 0.9058094385676062, 'bagging_fraction': 0.854796113118251, 'lambda_l1': 0.36712258621895766, 'lambda_l2': 74.2155855394315}. Best is trial 38 with value: 0.7755235817655564.


In [43]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

auc(best)=0.7755


{'num_leaves': 11,
 'min_child_samples': 92,
 'min_sum_hessian_in_leaf': 0.0008444201417052886,
 'feature_fraction': 0.8258337749307532,
 'bagging_fraction': 0.9117114344224849,
 'lambda_l1': 0.4706890247530332,
 'lambda_l2': 0.4314491087524046}

In [44]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 11,
 'min_child_samples': 92,
 'min_sum_hessian_in_leaf': 0.0008444201417052886,
 'feature_fraction': 0.8258337749307532,
 'bagging_fraction': 0.9117114344224849,
 'lambda_l1': 0.4706890247530332,
 'lambda_l2': 0.4314491087524046,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'learning_rate': 0.05,
 'n_estimators': 100000,
 'bagging_freq': 1}

In [45]:
train_oof, imp, metrics = train_lgb(
    input_x=x_train,
    input_y=y_train,
    input_id=id_train,
    params=params_best,
)

-------------------- 0 --------------------
(246008, 162) (61503, 162)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1282]	training's auc: 0.850067	valid_1's auc: 0.775524
[auc] tr:0.8501, va:0.7755
-------------------- 1 --------------------
(246009, 162) (61502, 162)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[821]	training's auc: 0.827195	valid_1's auc: 0.778412
[auc] tr:0.8272, va:0.7784
-------------------- 2 --------------------
(246009, 162) (61502, 162)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1128]	training's auc: 0.842601	valid_1's auc: 0.776221
[auc] tr:0.8426, va:0.7762
-------------------- 3 --------------------
(246009, 162) (61502, 162)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1036]	training's auc: 0.83959	valid_1's auc: 0.774899
[auc] tr:0.8396, va:0.774

In [46]:
x_test = df_test.drop(columns=["SK_ID_CURR"])
id_test = df_test[["SK_ID_CURR"]]

for col in x_test.columns:
    if x_test[col].dtype == "O":
        x_test[col] = x_test[col].astype("category")

In [47]:
test_pred = predict_lbg(
    x_test,
    id_test,
)

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [48]:
df_submit = test_pred.rename(columns={"pred": "TARGET"})
print(df_submit.shape)
display(df_submit.head())

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.032607
1,100005,0.134704
2,100013,0.024810
3,100028,0.047517
4,100038,0.201831


In [49]:
df_submit.to_csv("submission_HyperParameterTuning.csv", index=None)